In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, PolynomialFeatures,scale
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, LinearRegression,Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.compose import make_column_transformer, ColumnTransformer, make_column_selector
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer,IterativeImputer
from category_encoders.target_encoder import TargetEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import make_pipeline,Pipeline

In [11]:
from transformers import pipeline

In [12]:
path1 = "/Users/racheltan/Desktop/QMSS/AML/winemag-data-130k-v2.csv"
path2 = "/Users/ellen/Documents/GitHub/assignment-4-rachel_ellen/wine-reviews/winemag-data-130k-v2.csv"
wine_raw = pd.read_csv(path2, index_col = 0).drop(['taster_name', 'taster_twitter_handle'], axis = 1)

In [13]:
wine_usa = wine_raw[wine_raw.country == 'US']
wine_usa_skew = wine_usa[wine_usa.price < 250].drop_duplicates('description') #using wine only from US, remove duplicate rows and outliers

In [14]:
wine_us = wine_usa_skew.sample(n = 2000, random_state = 123) #subsample 20k rows 

In [15]:
wine_data = wine_us[['description', 'title','points']] 

In [16]:
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(
    wine_data['description'], wine_data['points'], stratify=wine_data['points'], random_state=0) #try just with the description column first

In [17]:
X_text_train = X_text_train.reset_index().drop('index',axis=1)    #when do this - only get 1 result for X_train below? 
y_text_train = y_text_train.reset_index().drop('index',axis=1)
X_text_test = X_text_test.reset_index().drop('index',axis=1)
y_text_test = y_text_test.reset_index().drop('index',axis=1)

In [18]:
X_text_train.head()

,description
0,"This mix of 68% Syrah, 20% Grenache and 12% Pe..."
1,"Very little Auxerrois is grown in Oregon, and ..."
2,This Chardonnay was blended from several viney...
3,This wine is spicy in forest floor and sage. S...
4,"This is so smooth, rich and mellow, you hardly..."


Trying out the BERT model

In [20]:
nlp = pipeline("feature-extraction")
 #using a pretrained BERT model to do the feature extraction 

In [34]:
X_train = [nlp(t) for t in X_text_train['description']]

In [59]:
X_train_1 = [np.array(X_train[i])[0].flatten() for i in range(len(X_train))]

In [64]:
X_train_1   #tried to use workflow from task 2 but made it weirder ???

[array([ 0.14785919, -0.16113544,  0.10633638, ...,  0.02205651,
         0.83032101,  0.13428324]),
 array([ 0.39385214, -0.08705731,  0.06369533, ..., -0.55039233,
         0.22575696,  0.64018559]),
 array([ 0.25697222, -0.09194963,  0.01049076, ...,  0.16141486,
         0.71054041,  0.30680642]),
 array([ 0.18590115, -0.20963667,  0.1418855 , ..., -0.56367302,
         0.56739569,  0.32316402]),
 array([ 0.22748159, -0.06811456, -0.06030596, ..., -0.3892062 ,
         0.77012998,  0.33098084]),
 array([ 0.15936592, -0.17225456,  0.21594357, ..., -0.04298722,
         0.76298696,  0.22870262]),
 array([0.27525762, 0.02873165, 0.02549801, ..., 0.14995067, 0.70530802,
        0.38630852]),
 array([ 0.17628577, -0.1412209 ,  0.10910372, ..., -0.13250004,
         1.05740106,  0.34990293]),
 array([ 0.29280457, -0.09248097,  0.10146098, ..., -0.28624567,
         0.73345321,  0.42645732]),
 array([ 0.24471967, -0.12063965,  0.10626665, ...,  0.24885607,
         0.392919  ,  0.05981323

In [65]:
score_bert_ridge = cross_val_score(Ridge(), X_train_1, y_text_train)
np.mean(score_bert_ridge) #not sure why this is not working? 

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: setting an array element with a sequence.

  FitFailedWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: setting an array element with a sequence.

  FitFailedWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: setting an array element with a sequence.

  FitFailedWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score

nan

In [ ]:
#for both columns text and description (to try after we figure out the other one)
nlp = pipeline("feature-extraction")
bert_nlp = nlp(t.decode())
preprocess = make_column_transformer(    
    (bert_nlp, 'description'), 
    (preprocess, 'title'))
bert_ridge = make_pipeline(preprocess, Ridge(alpha = 1.0))
score_bert_ridge = cross_val_score(bert_ridge, X_text_train, y_text_train)
np.mean(score_bert_ridge) 